## Depedências

In [5]:
# !nvidia-smi
# !pip install --upgrade dataframe-image
# !pip install kaleido
# !apt install firefox firefox-geckodriver
# !pip install dataframe_image selenium
# !apt install libgraphviz-dev
# !pip install pygraphviz

In [6]:
import kagglehub
import random
import json
import pickle
import os
import itertools
import keras
import pydot
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx
import matplotlib.pyplot as plt
# import dataframe_image as dfi
from networkx.drawing.nx_agraph import pygraphviz_layout
from PIL import Image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Concatenate, Add, Softmax, Multiply
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, BatchNormalization
from tensorflow.keras.initializers import he_normal
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard, CSVLogger, EarlyStopping
from tensorflow.keras.utils import get_file
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.base import BaseEstimator, ClassifierMixin
from joblib import Parallel, delayed
from collections import defaultdict
from scipy import stats
from sklearn.model_selection import train_test_split, KFold
from matplotlib import pyplot as plt

# Relations for Fashion MNIST
c2_to_c1 = {
    0:0, 1:0,
    2:1, 3:1,
    4:2, 5:2
}

# Nível da árvore 2 para 3 (folhas)
fine_to_c2 = {
    0:0, 1:0, 2:0,
    3:1, 4:1, 5:1,
    6:2, 7:2, 8:2,
    9:3, 10:3, 11:3,
    12:4, 13:4, 14:4,
    15:5, 16:5, 17:5,
}

unique_fine = [*range(0, 18, 1)]
unique_c2 = np.vectorize(fine_to_c2.get)(unique_fine)
unique_c1 = np.vectorize(c2_to_c1.get)(unique_c2)

relations = []
for i in range(18):
  relations.append([unique_c1[i],unique_c2[i]+2,unique_fine[i]+8])

# Computes hierarchical metrics
def hierarchical_metrics(true,pred):
  true_labels = []
  true_fine = true[2].argmax(axis=1)+8
  true_c2 = true[1].argmax(axis=1)+2
  true_c1 = true[0].argmax(axis=1)
  for i in range(len(true_fine)):
    true_labels.append([true_c1[i],true_c2[i],true_fine[i]])
  pred_labels = []
  pred_c1 = pred[0].argmax(axis = 1)
  pred_c2 = pred[1].argmax(axis = 1)+2
  pred_fine = pred[2].argmax(axis = 1)+8
  for i in range(len(pred_c1)):
    pred_labels.append([pred_c1[i],pred_c2[i],pred_fine[i]])
  preci = precision(true_labels,pred_labels)
  reca = recall(true_labels,pred_labels)
  f_1 = f1(true_labels,pred_labels)

  consistent_examples = 1
  correct_pred = 0
  test_set_size = len(true_labels)
  for i in range(test_set_size):
    if [pred_c1[i],pred_c2[i],pred_fine[i]] in relations:
        consistent_examples = consistent_examples + 1
    if [pred_c1[i],pred_c2[i],pred_fine[i]] == true_labels[i]:
        correct_pred = correct_pred +1
  h_accuracy = correct_pred/test_set_size
  h_consistency = (consistent_examples-1)/test_set_size

  return h_accuracy,h_consistency,f_1

# Hierarchical metrics, proposed by Kiritchenko et al (2005)
# Implementation
# https://gitlab.com/dacs-hpi/hiclass/-/blob/main/hiclass/metrics.py


def precision(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Compute precision score for hierarchical classification.

    hP = sum(|S intersection T|) / sum(|S|),
    where S is the set consisting of the most specific class(es) predicted for a test example and all respective ancestors
    and T is the set consisting of the true most specific class(es) for a test example and all respective ancestors.

    Parameters
    ----------
    y_true : np.array of shape (n_samples, n_levels)
        Ground truth (correct) labels.
    y_pred : np.array of shape (n_samples, n_levels)
        Predicted labels, as returned by a classifier.
    Returns
    -------
    precision : float
        What proportion of positive identifications was actually correct?
    """
    assert len(y_true) == len(y_pred)
    sum_intersection = 0
    sum_prediction_and_ancestors = 0
    for ground_truth, prediction in zip(y_true, y_pred):
        sum_intersection = sum_intersection + len(
            set(ground_truth).intersection(set(prediction))
        )
        sum_prediction_and_ancestors = sum_prediction_and_ancestors + len(
            set(prediction)
        )
    precision = sum_intersection / sum_prediction_and_ancestors
    return precision


def recall(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Compute recall score for hierarchical classification.

    hR = sum(|S intersection T|) / sum(|T|),
    where S is the set consisting of the most specific class(es) predicted for a test example and all respective ancestors
    and T is the set consisting of the true most specific class(es) for a test example and all respective ancestors.

    Parameters
    ----------
    y_true : np.array of shape (n_samples, n_levels)
        Ground truth (correct) labels.
    y_pred : np.array of shape (n_samples, n_levels)
        Predicted labels, as returned by a classifier.
    Returns
    -------
    recall : float
        What proportion of actual positives was identified correctly?
    """
    assert len(y_true) == len(y_pred)
    sum_intersection = 0
    sum_prediction_and_ancestors = 0
    for ground_truth, prediction in zip(y_true, y_pred):
        sum_intersection = sum_intersection + len(
            set(ground_truth).intersection(set(prediction))
        )
        sum_prediction_and_ancestors = sum_prediction_and_ancestors + len(
            set(ground_truth)
        )
    recall = sum_intersection / sum_prediction_and_ancestors
    return recall


def f1(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Compute f1 score for hierarchical classification.

    hF = 2 * hP * hR / (hP + hR),
    where hP is the hierarchical precision and hR is the hierarchical recall.

    Parameters
    ----------
    y_true : np.array of shape (n_samples, n_levels)
        Ground truth (correct) labels.
    y_pred : np.array of shape (n_samples, n_levels)
        Predicted labels, as returned by a classifier.
    Returns
    -------
    f1 : float
        Weighted average of the precision and recall
    """
    assert len(y_true) == len(y_pred)
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    f1 = 2 * prec * rec / (prec + rec)
    return f1

## Configurações Gerais

In [9]:
# Baixa última versão do dataset
path = kagglehub.dataset_download("paramaggarwal/fashion-product-images-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fashion-product-images-dataset' dataset.
Path to dataset files: /kaggle/input/fashion-product-images-dataset


In [10]:
# Carrega o arquivo de metadados
styles_csv_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/styles.csv'
styles_df = pd.read_csv(styles_csv_path, on_bad_lines='skip')

#----------------------------Primeira filtragem pelo masterCategory----------------------------
# Conta as ocorrências da categoria master
master_category_counts = styles_df['masterCategory'].value_counts()

# Pega as 3 maiores
top_3_master_categories = master_category_counts.nlargest(3).index.tolist()

# Filtra pelas 3 maiores
filtered_styles_df = styles_df[styles_df['masterCategory'].isin(top_3_master_categories)]
#----------------------------Primeira filtragem----------------------------

#----------------------------Segunda filtragem pelo subCategory----------------------------
top_2_subcategories_per_master = {}
filtered_styles_df_by_top_subcategories = pd.DataFrame()

for master_cat in top_3_master_categories:
    # Filtra o dataframe por categoria principal
    df_master_cat = styles_df[styles_df['masterCategory'] == master_cat]

    # Obtenha as 5 subcategorias mais frequentes
    top_sub_categories_all = df_master_cat['subCategory'].value_counts().nlargest(5).index.tolist()

    # Remove a indesejada "Watches"
    top_sub_categories = [sc for sc in top_sub_categories_all if sc != "Watches"][:2]

    top_2_subcategories_per_master[master_cat] = top_sub_categories

    # Filtra pelas 2 subcategorias escolhidas
    df_filtered_sub = df_master_cat[df_master_cat['subCategory'].isin(top_sub_categories)]
    filtered_styles_df_by_top_subcategories = pd.concat([filtered_styles_df_by_top_subcategories, df_filtered_sub])
# {'Apparel': ['Topwear', 'Bottomwear'], 'Accessories': ['Bags', 'Jewellery'], 'Footwear': ['Shoes', 'Sandal']}
#----------------------------Segunda filtragem pelo subCategory----------------------------

#----------------------------Terceira filtragem----------------------------
top_3_article_types_per_subcategory = {}
dataset = pd.DataFrame()

for sub_cat in filtered_styles_df_by_top_subcategories['subCategory'].unique():
    if sub_cat in ['Watches', 'Flip Flops']:
            continue  # ignora essas subcategorias

    df_sub_cat = filtered_styles_df_by_top_subcategories[filtered_styles_df_by_top_subcategories['subCategory'] == sub_cat]

    top_article_types = df_sub_cat['articleType'].value_counts().nlargest(3).index.tolist()
    top_3_article_types_per_subcategory[sub_cat] = top_article_types

    df_filtered_article = df_sub_cat[df_sub_cat['articleType'].isin(top_article_types)]

    dataset = pd.concat([dataset, df_filtered_article])
# {'Topwear': ['Tshirts', 'Shirts', 'Kurtas'], 'Bottomwear': ['Jeans', 'Shorts', 'Trousers'], 'Bags': ['Handbags', 'Backpacks', 'Clutches'], 'Jewellery': ['Earrings', 'Pendant', 'Necklace and Chains'], 'Shoes': ['Casual Shoes', 'Sports Shoes', 'Heels'], 'Sandal': ['Sandals', 'Sports Sandals', 'Flip Flops']}
#----------------------------Terceira filtragem----------------------------

In [11]:
## Dimensões da imagem
height, width = 32, 32

channel = 3
if K.image_data_format() == 'channels_first':
    input_shape = (channel, height, width)
else:
    input_shape = (height, width, channel)

In [12]:
# Quantidade de classes no primeiro nível hierárquico camada
coarse1_classes = 3

# Quantidade de classes no segundo nível hierárquico camada
coarse2_classes = 6

# Quantidade de classes no último nível hierárquico (folhas da árvore)
num_classes  = 18

# batch_size   = 128
# epochs       = 100

train_df, test_df = train_test_split(
    dataset,
    test_size=0.2,
    random_state=42,
)

In [13]:
def scheduler(epoch):
  learning_rate_init = 0.001
  if epoch > 55:
    learning_rate_init = 0.0002
  if epoch > 70:
    learning_rate_init = 0.00005
  return learning_rate_init

In [14]:
class LossWeightsModifier(keras.callbacks.Callback):
  def __init__(self, alpha, beta, gamma):
    self.alpha = alpha
    self.beta = beta
    self.gamma = gamma
  def on_epoch_end(self, epoch, logs={}):
    if epoch == 15:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.8)
      K.set_value(self.gamma, 0.1)
    if epoch == 25:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.2)
      K.set_value(self.gamma, 0.7)
    if epoch == 35:
      K.set_value(self.alpha, 0)
      K.set_value(self.beta, 0)
      K.set_value(self.gamma, 1)

In [15]:
# Classes do primeiro nível
class_names_c1 = ['Apparel', 'Accessories','Footwear']
c1_to_index = {name: idx for idx, name in enumerate(class_names_c1)}

# Classes do segundo nível
class_names_c2 = ['Topwear', 'Bottomwear', 'Bags', 'Jewellery', 'Shoes', 'Sandal',]
c2_to_index = {name: idx for idx, name in enumerate(class_names_c2)}

# Classes nas folhas
class_names =[
  "Tshirts",
  "Shirts",
  "Kurtas",
  "Jeans",
  "Shorts",
  "Trousers",
  "Handbags",
  "Backpacks",
  "Clutches",
  "Earrings",
  "Pendant",
  "Necklace and Chains",
  "Casual Shoes",
  "Sports Shoes",
  "Heels",
  "Sandals",
  "Sports Sandals",
  "Flip Flops",
]

fine_to_index = {class_name: index for index, class_name in enumerate(class_names)}

def print_mappings(mapping, source, dest):
    for k,v in mapping.items():
        print(source[k], "->", dest[v])

print_mappings(c2_to_c1, class_names_c2, class_names_c1)
print("-"*10)
print_mappings(fine_to_c2, class_names, class_names_c2)

Topwear -> Apparel
Bottomwear -> Apparel
Bags -> Accessories
Jewellery -> Accessories
Shoes -> Footwear
Sandal -> Footwear
----------
Tshirts -> Topwear
Shirts -> Topwear
Kurtas -> Topwear
Jeans -> Bottomwear
Shorts -> Bottomwear
Trousers -> Bottomwear
Handbags -> Bags
Backpacks -> Bags
Clutches -> Bags
Earrings -> Jewellery
Pendant -> Jewellery
Necklace and Chains -> Jewellery
Casual Shoes -> Shoes
Sports Shoes -> Shoes
Heels -> Shoes
Sandals -> Sandal
Sports Sandals -> Sandal
Flip Flops -> Sandal


In [16]:
#----------Pega os pesos do modelo VGG16 pré-treinado--------
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
weights_path = get_file(
    'vgg16_weights_tf_dim_ordering_tf_kernels.h5',
    WEIGHTS_PATH,
    cache_subdir='models',
)

# Carregando os dados caso não tenha salvo anteriormente
train_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/images/'

# Inicializa listas de dados
x_train, y_train = [], []
x_test, y_test = [], []

# Carrega os dados salvos
with open('dados_32x32.pkl', 'rb') as f:
    data = pickle.load(f)
    x_train = data['x_train']
    y_train = data['y_train']
    x_test = data['x_test']
    y_test = data['y_test']

# One Hot Encondig para as labels fine
train_labels_fine = to_categorical(y_train)
test_labels_fine = to_categorical(y_test)

# One Hot Encoding para as labels da segunda camada grossa
train_labels_c2_index = [fine_to_c2[i] for i in y_train]
train_labels_c2 = to_categorical(train_labels_c2_index)

test_labels_c2_index = [fine_to_c2[i] for i in y_test]
test_labels_c2 = to_categorical(test_labels_c2_index)

# One Hot Enconding para as labels da primeira camada grossa
train_labels_c1_index = [c2_to_c1[i] for i in train_labels_c2_index]
train_labels_c1 = to_categorical(train_labels_c1_index)

test_labels_c1_index = [c2_to_c1[i] for i in test_labels_c2_index]
test_labels_c1 = to_categorical(test_labels_c1_index)

# Finalmente criando as variáveis labels para treino, teste e validação
y_train = train_labels_fine
y_test = test_labels_fine

y_c1_train = train_labels_c1
y_c1_test = test_labels_c1

y_c2_train = train_labels_c2
y_c2_test = test_labels_c2

# Coloca o Channel por último
x_train = x_train.transpose(0, 2, 3, 1)
x_test = x_test.transpose(0, 2, 3, 1)

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

print("y_c1_train shape: ", y_c1_train.shape)
print("y_c1_test shape: ", y_c1_test.shape)

print("y_c2_train shape: ", y_c2_train.shape)
print("y_c2_test shape: ", y_c2_test.shape)

# Transforma o id das imagens para indices
# id_to_index = {image_id: index for index, image_id in enumerate(dataset['id'])}

# # Loop para carregar imagens com shape fixo (3, 32, 32)
# for image_id in dataset['id']:
#     image_filename = str(image_id) + '.jpg'
#     image_path = os.path.join(train_path, image_filename)

#     if os.path.exists(image_path):
#         dataset_index = id_to_index[image_id]

#         img = Image.open(image_path).convert("RGB")  # Garante 3 canais
#         img = img.resize((width, height))
#         img_array = np.array(img)

#         if image_id in train_df['id'].values:
#             x_train.append(img_array)
#             y_train.append(fine_to_index[dataset.iloc[dataset_index]['articleType']])

#         elif image_id in test_df['id'].values:
#             x_test.append(img_array)
#             y_test.append(fine_to_index[dataset.iloc[dataset_index]['articleType']])

# x_train = np.array(x_train).transpose(0, 3, 1, 2).astype("float32")
# x_test = np.array(x_test).transpose(0, 3, 1, 2).astype("float32")

# x_train = (x_train - np.mean(x_train)) / np.std(x_train)
# x_test = (x_test - np.mean(x_test)) / np.std(x_test)

# y_train = np.array(y_train)
# y_test = np.array(y_test)

# with open("dados_32x32.pkl", "wb") as f:
#     pickle.dump(
#         {
#             "x_train": x_train,
#             "x_test": x_test,
#             "y_train": y_train,
#             "y_test": y_test
#         },
#         f
#     )


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 22s 0us/step
x_train shape:  (19594, 32, 32, 3)
x_test shape:  (4900, 32, 32, 3)
y_train shape:  (19594, 18)
y_test shape:  (4900, 17)
y_c1_train shape:  (19594, 3)
y_c1_test shape:  (4900, 3)
y_c2_train shape:  (19594, 6)
y_c2_test shape:  (4900, 6)


# BA-CNN

In [17]:
# Best Hypeparameters
# if True, the model uses BT-strategy for training
# bt_strategy = True

# neurons of all dense layers on each branch
# branch_neurons = 16    # Parsimonious version 256

# att_neurons = 2048     # Parsimonious version 64

In [18]:
def create_model(
  optimizer="sgd",
  learning_rate=0.001,
  momentum=0.9,
  nesterov=True,
  bt_strategy=True,
  branch_neurons=128,
  att_neurons=128,
):
  # ----------------------- Otimizador ---------------------------
  if optimizer.lower() == "sgd":
      opt = optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=nesterov)
  elif optimizer.lower() == "adam":
      opt = optimizers.Adam(learning_rate=learning_rate)
  else:
      raise ValueError("Otimizador não suportado")

  # ----------------------- Loss Weights -------------------------
  if bt_strategy:
      alpha = K.variable(0.98, dtype="float32", name="alpha")
      beta  = K.variable(0.01, dtype="float32", name="beta")
      gamma = K.variable(0.01, dtype="float32", name="gamma")
  else:
      alpha = K.variable(0.33, dtype="float32", name="alpha")
      beta  = K.variable(0.33, dtype="float32", name="beta")
      gamma = K.variable(0.34, dtype="float32", name="gamma")

  # ----------------------- Modelo ------------------------------
  img_input = Input(shape=input_shape, name='input')

  # Block 1
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2), strides=(2, 2))(x)

  # Block 2
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2), strides=(2, 2))(x)

  # Coarse 1
  c_1_bch = Flatten(name='c1_flatten')(x)
  c_1_bch = Dense(branch_neurons, activation='relu')(c_1_bch)
  c_1_bch = BatchNormalization()(c_1_bch)
  c_1_bch = Dropout(0.5)(c_1_bch)
  c_1_bch = Dense(branch_neurons, activation='relu')(c_1_bch)
  c_1_bch = BatchNormalization()(c_1_bch)
  c_1_bch = Dropout(0.5)(c_1_bch)
  #c_1_pred = Dense(coarse1_classes, activation='softmax')(c_1_bch)

  # Block 3
  x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2), strides=(2, 2))(x)

  # Coarse 2
  c_2_bch = Flatten(name='c2_flatten')(x)
  c_2_bch = Dense(branch_neurons, activation='relu')(c_2_bch)
  c_2_bch = BatchNormalization()(c_2_bch)
  c_2_bch = Dropout(0.5)(c_2_bch)
  c_2_bch = Dense(branch_neurons, activation='relu')(c_2_bch)
  c_2_bch = BatchNormalization()(c_2_bch)
  c_2_bch = Dropout(0.5)(c_2_bch)

  # Block 4
  x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2), strides=(2, 2))(x)

  # Block 5
  x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)

  # Fine
  x = Flatten(name='flatten')(x)
  x = Dense(branch_neurons, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.5)(x)
  x = Dense(branch_neurons, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.5)(x)

  #-- Att for coarse 1---
  # Coarse 1
  sfcn_1_1 = Dense(att_neurons, name='fc1_1')(c_1_bch)
  sfcn_1_1 = Dense(1, name='fc1_2')(sfcn_1_1)
  # Coarse 2
  sfcn_1_2 = Dense(att_neurons, name='fc1_3')(c_2_bch)
  sfcn_1_2 = Dense(1, name='fc1_4')(sfcn_1_2)
  # Fine
  sfcn_1_3 = Dense(att_neurons, name='fc1_5')(x)
  sfcn_1_3 = Dense(1, name='fc1_6')(sfcn_1_3)

  score_vector_1 = Concatenate()([sfcn_1_1,sfcn_1_2,sfcn_1_3]) # Score vector 1
  att_weights_1 = Activation('softmax', name='attention_weights_1')(score_vector_1) # Attention weights 1

  weighted_c1_bch_1 = Multiply()([c_1_bch, att_weights_1[:, 0:1]])
  weighted_c2_bch_1 = Multiply()([c_2_bch, att_weights_1[:, 1:2]])
  weighted_x_1 = Multiply()([x, att_weights_1[:, 2:3]])

  weightned_sum_1 = Add()([weighted_c1_bch_1, weighted_c2_bch_1, weighted_x_1]) # Weightned sum 1

  # Concat and prediction
  coarse_1_concat = Concatenate()([c_1_bch,weightned_sum_1])
  c_1_pred = Dense(coarse1_classes, activation='softmax', name='c1_predictions_clothes')(coarse_1_concat)

  #-- Att for coarse 2---

  # Coarse 1
  sfcn_2_1 = Dense(att_neurons, name='fc2_1')(c_1_bch)
  sfcn_2_1 = Dense(1, name='fc2_2')(sfcn_2_1)
  # Coarse 2
  sfcn_2_2 = Dense(att_neurons, name='fc2_3')(c_2_bch)
  sfcn_2_2 = Dense(1, name='fc2_4')(sfcn_2_2)
  # Fine
  sfcn_2_3 = Dense(att_neurons, name='fc2_5')(x)
  sfcn_2_3 = Dense(1, name='fc2_6')(sfcn_2_3)

  score_vector_2 = Concatenate()([sfcn_2_1,sfcn_2_2,sfcn_2_3]) # Score vector 1
  att_weights_2 = Activation('softmax', name='attention_weights_2')(score_vector_2) # Attention weights 1

  weighted_c1_bch_2 = Multiply()([c_1_bch, att_weights_2[:, 0:1]])
  weighted_c2_bch_2 = Multiply()([c_2_bch, att_weights_2[:, 1:2]])
  weighted_x_2 = Multiply()([x, att_weights_2[:, 2:3]])

  weightned_sum_2 = Add()([weighted_c1_bch_2, weighted_c2_bch_2, weighted_x_2]) # Weightned sum 1

  # Concat and prediction
  coarse_2_concat = Concatenate()([c_2_bch,weightned_sum_2])
  c_2_pred = Dense(coarse2_classes, activation='softmax', name='c2_predictions_clothes')(coarse_2_concat)

  #-- Att for fine---
  # Coarse 1
  sfcn_3_1 = Dense(att_neurons, name='fc3_1')(c_1_bch)
  sfcn_3_1 = Dense(1, name='fc3_2')(sfcn_3_1)
  # Coarse 2
  sfcn_3_2 = Dense(att_neurons, name='fc3_3')(c_2_bch)
  sfcn_3_2 = Dense(1, name='fc3_4')(sfcn_3_2)
  # Fine
  sfcn_3_3 = Dense(att_neurons, name='fc3_5')(x)
  sfcn_3_3 = Dense(1, name='fc3_6')(sfcn_3_3)

  score_vector_3 = Concatenate()([sfcn_3_1,sfcn_3_2,sfcn_3_3]) # Score vector 1
  att_weights_3 = Activation('softmax', name='attention_weights_3')(score_vector_3) # Attention weights 1

  weighted_c1_bch_3 = Multiply()([c_1_bch, att_weights_3[:, 0:1]])
  weighted_c2_bch_3 = Multiply()([c_2_bch, att_weights_3[:, 1:2]])
  weighted_x_3 = Multiply()([x, att_weights_3[:, 2:3]])

  weightned_sum_3 = Add()([weighted_c1_bch_3, weighted_c2_bch_3, weighted_x_3]) # Weightned sum 3

  # Concat and prediction
  fine_concat = Concatenate()([x,weightned_sum_3])
  fine_pred = Dense(num_classes, activation='softmax', name='predictions_clothes')(fine_concat)

  model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='bacnn')

  # Callbacks
  change_lr = LearningRateScheduler(scheduler)
  change_lw = LossWeightsModifier(alpha, beta, gamma)

  if bt_strategy == True:
    cbks = [change_lr, change_lw]
  else:
    cbks = [change_lr]

  #----------------------- compile ---------------------------
  # sgd = optimizers.SGD(learning_rate=0.0002, momentum=0.9, nesterov=True)
  model.compile(
    loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'],
    optimizer=opt,
    loss_weights=[alpha, beta, gamma],
    metrics=['accuracy', 'accuracy', 'accuracy'],
  )

  model._cbks = cbks
  # model.summary()

  return model

In [ ]:
# Grid Search
# ======== DEFINIÇÃO DO GRID ==========
param_grid = {
    'batch_size': [64, 128],
    'branch_neurons': [16, 32, 64],
    'att_neurons': [2048, 4056],
    'learning_rate_init': [0.001, 0.0001],
    'momentum': [0.9, 0.95],
    'optimizer': ["sgd", "adam"],
    'nesterov': [True, False],
    'bt_strategy': [True, False]
}

# ======== CALLBACKS ==========
early_stop = EarlyStopping(
    monitor='val_predictions_clothes_loss',  # ajuste conforme o nome da sua saída
    patience=10,
    restore_best_weights=True,
    mode='min'
)

# ======== GERAR TODAS AS COMBINAÇÕES ==========
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# ======== CONFIGURAÇÃO ==========
chunk_size = 50  # quantas combinações rodar por execução
checkpoint_file = "checkpoint.txt"
results_file = "results.json"

# ======== RETOMADA DO PONTO ==========
start_idx = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        start_idx = int(f.read().strip())

print(f"Retomando do índice {start_idx}/{len(combinations)}")

# ======== CARREGAR RESULTADOS JÁ EXISTENTES ==========
results = []
if os.path.exists(results_file):
    with open(results_file, "r") as f:
        results = json.load(f)

best_score = max([r["score"] for r in results], default=-np.inf)
best_params = None

# ======== LOOP NAS COMBINAÇÕES ==========
end_idx = min(start_idx + chunk_size, len(combinations))
for i in range(start_idx, end_idx):
    params = combinations[i]
    print(f"\nTreinando combinação {i+1}/{len(combinations)}: {params}")

    # Criar o modelo
    model = create_model(
        optimizer=params['optimizer'],
        learning_rate=params['learning_rate_init'],
        momentum=params['momentum'],
        nesterov=params['nesterov'],
        bt_strategy=params['bt_strategy'],
        branch_neurons=params['branch_neurons'],
        att_neurons=params['att_neurons']
    )

    # Treinar
    history = model.fit(
        x_train,
        [y_c1_train, y_c2_train, y_train],
        batch_size=params['batch_size'],
        epochs=100,
        verbose=1,
        validation_split=0.1,  # 20% do treino para validação interna
        callbacks=[early_stop]
    )

    # Previsões no conjunto de teste
    predictions = model.predict(x_test, verbose=1)

    # Calcular métricas hierárquicas
    acc_ba_cnn_c, cons_ba_cnn_c, f1_ba_cnn_c = hierarchical_metrics(
        [y_c1_test, y_c2_test, y_test],
        predictions
    )

    score = f1_ba_cnn_c

    # Salvar resultados
    result_entry = {
        'params': params,
        'acc': acc_ba_cnn_c,
        'cons': cons_ba_cnn_c,
        'f1': f1_ba_cnn_c,
        'score': score
    }
    results.append(result_entry)

    if score > best_score:
        best_score = score
        best_params = params

    print(f"  -> Resultados holdout: acc={acc_ba_cnn_c:.4f}, cons={cons_ba_cnn_c:.4f}, f1={f1_ba_cnn_c:.4f}, score={score:.4f}")

    # ======== SALVAR PROGRESSO ==========
    with open(results_file, "w") as f:
        json.dump(results, f, indent=4)

    with open(checkpoint_file, "w") as f:
        f.write(str(i+1))  # próximo índice a ser processado

print("\n=== GRID SEARCH PARCIAL FINALIZADO ===")
print("Melhor score até agora:", best_score)
print("Melhores parâmetros até agora:", best_params)
print(f"Treinou do índice {start_idx} até {end_idx-1}")

# history_bcnn = model.fit(
#     x_train,
#     [y_c1_train, y_c2_train, y_train],
#     batch_size=batch_size,
#     epochs=epochs,
#     verbose=1,
#     callbacks=cbks,
# )

# score_ba_cnn_c = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
# parameters_ba_cnn_c = np.sum([K.count_params(w) for w in model.trainable_weights])

# predictions = model.predict(x_test)
# acc_ba_cnn_c, cons_ba_cnn_c, f1_ba_cnn_c= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# # Save model
# model.save("ba_cnn_base.keras")

# # Save results to file
# with open('results_ba_cnn_base.txt', 'w') as f:  # New file for BA-CNN
#   f.write("--- BA-CNN Base C ---\n")
#   f.write(f"Accuracy level 1: {score_ba_cnn_c[4]}\n")
#   f.write(f"Accuracy level 2: {score_ba_cnn_c[5]}\n")
#   f.write(f"Accuracy level 3: {score_ba_cnn_c[6]}\n")
#   f.write(f"Hierarchical Accuracy: {acc_ba_cnn_c}\n")
#   f.write(f"Consistency: {cons_ba_cnn_c}\n")
#   f.write(f"F1: {f1_ba_cnn_c}\n")
#   f.write(f"Parameters: {parameters_ba_cnn_c:,}\n")

# # Results
# print("--- BA-CNN Base C ---")
# print("--- Accuracy per level ---")
# print("Accuracy level 1:",score_ba_cnn_c[4])
# print("Accuracy level 2:",score_ba_cnn_c[5])
# print("Accuracy level 3:",score_ba_cnn_c[6])
# print("--- Hierarchical Metrics ---")
# print("Accuracy:",acc_ba_cnn_c)
# print("Consistency:",cons_ba_cnn_c)
# print("f1:",f1_ba_cnn_c)
# print("Parameters:","{:,}".format(parameters_ba_cnn_c))

Retomando do índice 218/384

Treinando combinação 219/384: {'batch_size': 128, 'branch_neurons': 16, 'att_neurons': 2048, 'learning_rate_init': 0.0001, 'momentum': 0.95, 'optimizer': 'sgd', 'nesterov': False, 'bt_strategy': True}
Epoch 1/100
138/138 ━━━━━━━━━━━━━━━━━━━━ 62s 252ms/step - c1_predictions_clothes_accuracy: 0.5295 - c1_predictions_clothes_loss: 1.1561 - c2_predictions_clothes_accuracy: 0.2574 - c2_predictions_clothes_loss: 2.3217 - loss: 1.1925 - predictions_clothes_accuracy: 0.0603 - predictions_clothes_loss: 3.6330 - val_c1_predictions_clothes_accuracy: 0.9949 - val_c1_predictions_clothes_loss: 0.5658 - val_c2_predictions_clothes_accuracy: 0.0158 - val_c2_predictions_clothes_loss: 2.0950 - val_loss: 0.6061 - val_predictions_clothes_accuracy: 0.0000e+00 - val_predictions_clothes_loss: 2.9694
Epoch 2/100
138/138 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - c1_predictions_clothes_accuracy: 0.8044 - c1_predictions_clothes_loss: 0.4914 - c2_predictions_clothes_accuracy: 0.2676 - c2_pr

In [ ]:
# print(history_bcnn.history.keys())

plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
plt.plot(history_bcnn.history['c1_predictions_clothes_accuracy'])
plt.plot(history_bcnn.history['val_c1_predictions_clothes_accuracy'])
plt.title('Model Accuracy - Coarse 1')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(3, 1, 2)
plt.plot(history_bcnn.history['c2_predictions_clothes_accuracy'])
plt.plot(history_bcnn.history['val_c2_predictions_clothes_accuracy'])
plt.title('Model Accuracy - Coarse 2')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(3, 1, 3)
plt.plot(history_bcnn.history['predictions_clothes_accuracy'])
plt.plot(history_bcnn.history['val_predictions_clothes_accuracy'])
plt.title('Model Accuracy - Fine')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(12, 4))
plt.plot(history_bcnn.history['loss'])
plt.plot(history_bcnn.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Árvore Hierárquica

In [ ]:
# # Carrega o modelo
# model = load_model('ba_cnn_base.keras')

# # Analisa o que o modelo prediziu
# preds = model.predict(x_test, batch_size=32)

# c1_preds = np.argmax(preds[0], axis=1)
# c2_preds = np.argmax(preds[1], axis=1)
# fine_preds = np.argmax(preds[2], axis=1)

# Cria a estrutura do Grafo, nesse caso é um grafo unidirecional
G = nx.DiGraph()

# Adiciona os nós iniciais
G.add_node("Root")
G.add_node("Apparel")
G.add_node("Accessories")
G.add_node("Footwear")
G.add_node("Personal Care")

# Adiciona as ligações entre o nós raiz e os demais da primeira camada
G.add_edges_from([("Root", "Apparel"),("Root", "Accessories"),("Root", "Footwear"),("Root", "Personal Care")])

# Adiciona as ligações entre os nós da primeira camada com os da segunda camada
excluded_classes = ["Watches", "Flip Flops", "Bra"]
for c1_pred, c2_pred in zip(c1_preds, c2_preds):
    if class_names_c1[c1_pred] in excluded_classes or class_names_c2[c2_pred] in excluded_classes:
        continue
    G.add_edge(class_names_c1[c1_pred], class_names_c2[c2_pred])

# Adiciona as ligações entre os nós da segunda camada com os da terceira camada já com a ponderação
for c2_pred, fine_pred in zip(c2_preds, fine_preds):
    if class_names_c2[c2_pred] in excluded_classes or class_names[fine_pred] in excluded_classes:
        continue
    G.add_edge(class_names_c2[c2_pred], class_names[fine_pred])

In [ ]:
# Define o tamanhho da imagem
plt.figure(figsize=(60, 10))

edge_counts = defaultdict(int)

# prediction_paths = []
for c1_pred, c2_pred in zip(c1_preds, c2_preds):
    edge_counts[(class_names_c1[c1_pred], class_names_c2[c2_pred])] += 1

for c2_pred, fine_pred in zip(c2_preds, fine_preds):
    edge_counts[(class_names_c2[c2_pred], class_names[fine_pred])] += 1

# Define a layout do Grafo como uma árvore
pos = pygraphviz_layout(G, prog="dot")

# Desenha os nós da árvore
nx.draw_networkx_nodes(G, pos, node_color="#11479f")

# Desenha os nomes dos nós
nx.draw_networkx_labels(
    G,
    pos,
    font_color="white",
    font_weight="bold",
    bbox=dict(boxstyle="round,pad=0.3", fc="#11479f"),
)

# Desenha as conexões entre os nós
edges = list(G.edges())
alphas = []
max_count = max(edge_counts.values())

for edge in edges:
    if edge in [('Root', 'Apparel'), ('Root', 'Accessories'), ('Root', 'Footwear'), ('Root', 'Personal Care')]:
        alphas.append(1)
        continue

    alpha = 0.1 + 0.9 * (edge_counts[edge] / max_count)
    alphas.append(alpha)

nx.draw_networkx_edges(G, pos, edge_color="black", edgelist=edges, alpha=alphas, width=2)

# Plota a árvore
plt.show()

# # Salva a imagem na árvore em um arquivo
# plt.savefig("hierarchical_graph.png")

# Resultados

## BACNN

In [ ]:
folder_path = './results/bacnn'  # Change this to your folder path

data = []

patterns = {
    "Accuracy level 1": r"Accuracy level 1:\s+([0-9.]+)",
    "Accuracy level 2": r"Accuracy level 2:\s+([0-9.]+)",
    "Accuracy level 3": r"Accuracy level 3:\s+([0-9.]+)",
    "Hierarchical Accuracy": r"Hierarchical Accuracy:\s+([0-9.]+)",
    "Consistency": r"Consistency:\s+([0-9.]+)",
    "F1": r"F1:\s+([0-9.]+)",
    "Parameters": r"Parameters:\s+([\d,]+)"
}

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        filepath = os.path.join(folder_path, filename)
        with open(filepath, 'r') as file:
            content = file.read()
            row = {}
            for key, pattern in patterns.items():
                match = re.search(pattern, content)
                if match:
                    value = match.group(1)
                    # Convert numeric values
                    if key == "Parameters":
                        value = int(value.replace(",", ""))
                    elif key != "Model":
                        value = float(value)
                    row[key] = value

            epoch_match = re.search(r'_(\d+)_epochs', filename)
            row["Epochs"] = int(epoch_match.group(1)) if epoch_match else None

            data.append(row)

df = pd.DataFrame(data).sort_values(by="Epochs")
df = df.rename(columns={
    "Accuracy level 1": "Acc Level 1",
    "Accuracy level 2": "Acc Level 2",
    "Accuracy level 3": "Acc Level 3",
    "Hierarchical Accuracy": "Hierarchical Acc",
    "Parameters": "Params"
})

df = df.style.highlight_max(color='red', subset=['Acc Level 1', 'Acc Level 2',"Acc Level 3","Hierarchical Acc","Consistency","F1"]).hide(axis='index')

dfi.export(df, 'results_bacnn.png', table_conversion='selenium')

## HCNN

In [ ]:
folder_path = './results/hcnn_without_bt'

data = []

patterns = {
    "Accuracy level 1": r"Accuracy level 1:\s+([0-9.]+)",
    "Accuracy level 2": r"Accuracy level 2:\s+([0-9.]+)",
    "Accuracy level 3": r"Accuracy level 3:\s+([0-9.]+)",
    "Hierarchical Accuracy": r"Hierarchical Accuracy:\s+([0-9.]+)",
    "Consistency": r"Consistency:\s+([0-9.]+)",
    "F1": r"F1:\s+([0-9.]+)",
    "Parameters": r"Parameters:\s+([\d,]+)"
}

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        filepath = os.path.join(folder_path, filename)
        with open(filepath, 'r') as file:
            content = file.read()
            row = {}
            for key, pattern in patterns.items():
                match = re.search(pattern, content)
                if match:
                    value = match.group(1)
                    # Convert numeric values
                    if key == "Parameters":
                        value = int(value.replace(",", ""))
                    elif key != "Model":
                        value = float(value)
                    row[key] = value

            epoch_match = re.search(r'_(\d+)_epochs', filename)
            row["Epochs"] = int(epoch_match.group(1)) if epoch_match else None

            data.append(row)

# Create DataFrame
df = pd.DataFrame(data).sort_values(by="Epochs")
df = df.rename(columns={
    "Accuracy level 1": "Acc Level 1",
    "Accuracy level 2": "Acc Level 2",
    "Accuracy level 3": "Acc Level 3",
    "Hierarchical Accuracy": "Hierarchical Acc",
    "Parameters": "Params"
})

df = df.style.highlight_max(color='red', subset=['Acc Level 1', 'Acc Level 2',"Acc Level 3","Hierarchical Acc","Consistency","F1"]).hide(axis='index')

dfi.export(df, 'results_hcnn_without_bt.png', table_conversion='selenium')

## BCNN

In [ ]:
folder_path = './results/bcnn_without_bt'
data = []

patterns = {
    "Accuracy level 1": r"Accuracy level 1:\s+([0-9.]+)",
    "Accuracy level 2": r"Accuracy level 2:\s+([0-9.]+)",
    "Accuracy level 3": r"Accuracy level 3:\s+([0-9.]+)",
    "Hierarchical Accuracy": r"Hierarchical Accuracy:\s+([0-9.]+)",
    "Consistency": r"Consistency:\s+([0-9.]+)",
    "F1": r"F1:\s+([0-9.]+)",
    "Parameters": r"Parameters:\s+([\d,]+)"
}

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        filepath = os.path.join(folder_path, filename)
        with open(filepath, 'r') as file:
            content = file.read()
            row = {}
            for key, pattern in patterns.items():
                match = re.search(pattern, content)
                if match:
                    value = match.group(1)
                    # Convert numeric values
                    if key == "Parameters":
                        value = int(value.replace(",", ""))
                    elif key != "Model":
                        value = float(value)
                    row[key] = value

            epoch_match = re.search(r'_(\d+)_epochs', filename)
            row["Epochs"] = int(epoch_match.group(1)) if epoch_match else None

            data.append(row)

# Create DataFrame
df = pd.DataFrame(data).sort_values(by="Epochs")
df = df.rename(columns={
    "Accuracy level 1": "Acc Level 1",
    "Accuracy level 2": "Acc Level 2",
    "Accuracy level 3": "Acc Level 3",
    "Hierarchical Accuracy": "Hierarchical Acc",
    "Parameters": "Params"
})

df = df.style.highlight_max(color='red', subset=['Acc Level 1', 'Acc Level 2',"Acc Level 3","Hierarchical Acc","Consistency","F1"]).hide(axis='index')

dfi.export(df, 'results_bcnn_without_bt.png', table_conversion='selenium')